In [2]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Pruning added

In [2]:
import torch

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = df['numeric_label'].values.tolist()
        self.features = df.drop(columns=['numeric_label'], axis=1).values.tolist()

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_features(self, idx):
        return np.array(self.features[idx])

    def __getitem__(self, idx):
        batch_features = self.get_batch_features(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_features, batch_y

In [3]:
def build_model_custom(trial):
    
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 332
    
    for i in range(n_layers):
        
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 18)
        
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.LeakyReLU())

        in_features = out_features
        
    layers.append(nn.Linear(in_features, 3))
    layers.append(nn.LeakyReLU())
    
    return nn.Sequential(*layers)
 
 
def train_and_evaluate(param, model, trial):
    
    df = pd.read_csv('tweets_plus_emb.csv')
    df = df.drop(columns=['label'], axis=1)

    train_data, val_data = train_test_split(df, test_size = 0.2, random_state = 42)
    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=param['n_batch'], shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=param['n_batch'])

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr=param['learning_rate'])

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    EPOCHS = param['n_epochs']

    for epoch_num in range(EPOCHS):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in train_dataloader:

                train_label = train_label.to(device)
                train_input = train_input.to(device)

                output = model(train_input.float())
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    val_input = val_input.to(device)

                    output = model(val_input.float())

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            accuracy = total_acc_val/len(val_data)
            trial.report(accuracy, epoch_num)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    return accuracy
  
def objective(trial):

     params = {
              'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "AdamW", "RMSprop"]),
              'n_batch': trial.suggest_int("n_batch", 8, 32),
              'n_epochs': trial.suggest_int("n_epochs", 3, 8)
              }
    
     model = build_model_custom(trial)
     accuracy = train_and_evaluate(params, model, trial)

     return accuracy

In [18]:
import pso_sampler
import harmony_sampler
import simulated_annealing
from optuna.samplers import RandomSampler
import imp
imp.reload(harmony_sampler)
imp.reload(pso_sampler)
from optuna.samplers import NSGAIISampler

In [20]:
study = optuna.create_study(direction="maximize", sampler=simulated_annealing.SimulatedAnnealingSampler(), pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=50)

In [6]:
best_trial = study.best_trial

for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

#learning_rate: 0.01200409134216893
# optimizer: RMSprop
# n_batch: 8
# n_epochs: 7
# n_layers: 3
# n_units_l0: 15
# n_units_l1: 18
# n_units_l2: 9

# TPE max = Best is trial 12 with value: 0.6715807247694506
# learning_rate: 0.003139096261498651
# optimizer: Adam
# n_batch: 18
# n_epochs: 7
# n_layers: 3
# n_units_l0: 14
# n_units_l1: 4
# n_units_l2: 18

# NSGA max = Best is trial 45 with value: 0.6660605273412131
# learning_rate: 0.0049801250479125335
# optimizer: RMSprop
# n_batch: 19
# n_epochs: 4
# n_layers: 1
# n_units_l0: 18

# PSO max = Best is trial 32 with value: 0.6658007533445902
# learning_rate: 0.01328152397702899
# optimizer: Adam
# n_batch: 32
# n_epochs: 7
# n_layers: 1
# n_units_l0: 18

# harmony max = Best is trial 40 with value: 0.6711261202753604.
# harmony_memory_size=8, max_iter_size=50)
# learning_rate: 0.0027146341538599227
# optimizer: Adam
# n_batch: 15
# n_epochs: 5
# n_layers: 1
# n_units_l0: 16

# random search max = Best is trial 22 with value: 0.6663852448369918.
# learning_rate: 0.04541747213094801
# optimizer: RMSprop
# n_batch: 14
# n_epochs: 8
# n_layers: 1
# n_units_l0: 14

learning_rate: 0.04541747213094801
optimizer: RMSprop
n_batch: 14
n_epochs: 8
n_layers: 1
n_units_l0: 14


In [7]:
study.trials_dataframe().to_csv('dnn_results/random_50_trials_11_24_w_pruning.csv', index=False)

## Tuning Visualization

In [11]:
optuna.visualization.plot_intermediate_values(study)
# figure.write_image(f'dnn_results/plots/{algo}_50_trials_w_pruning.png')
# tpe_50_trials_w_pruning_intermed_values
# nsga_50_trials_w_pruning_intermed_values
# pso_50_trials_w_pruning_intermed_values
# harmony_50_trials_w_pruning_intermed_values
# siman_50_trials_w_pruning_intermed_values
# rnd_50_trials_w_pruning_intermed_values

In [12]:
optuna.visualization.plot_optimization_history(study)
# tpe_50_trials_w_pruning_opt_history
# nsga_50_trials_w_pruning_opt_history
# pso_50_trials_w_pruning_opt_history
# harmony_50_trials_w_pruning_opt_history
# siman_50_trials_w_pruning_opt_history
# rnd_50_trials_w_pruning_opt_history


In [14]:
optuna.visualization.plot_parallel_coordinate(study)
# tpe_50_trials_w_pruning_parall_coord
# nsga_50_trials_w_pruning_parall_coord
# pso_50_trials_w_pruning_parall_coord
# harmony_50_trials_w_pruning_parall_coord
# siman_50_trials_w_pruning_parall_coord
# rnd_50_trials_w_pruning_parall_coord


In [16]:
optuna.visualization.plot_param_importances(study)
# tpe_50_trials_w_pruning_param_imp
# nsga_50_trials_w_pruning_param_imp
# pso_50_trials_w_pruning_param_imp
# harmony_50_trials_w_pruning_param_imp
# siman_50_trials_w_pruning_param_imp
# rnd_50_trials_w_pruning_param_imp


In [15]:
from os import listdir

def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]
filenames = find_csv_filenames('dnn_results')
filenames

['simulated_annealing_50_trials_11_20_w_pruning.csv',
 'tpe_50_trials_11_20_w_pruning.csv',
 'random_50_trials_11_24_w_pruning.csv',
 'nsga_50_trials_11_20_w_pruning.csv',
 'pso_50_trials_11_20_w_pruning.csv',
 'harmony_50_trials_11_22_w_pruning.csv']

In [20]:
for f_name in filenames:
  df = pd.read_csv(f'dnn_results/{f_name}')
  print(f_name, (pd.Timestamp(df.iloc[-1]['datetime_complete']) - pd.Timestamp(df.iloc[0]['datetime_start'])))


simulated_annealing_50_trials_11_20_w_pruning.csv 0 days 00:14:27.283990
tpe_50_trials_11_20_w_pruning.csv 0 days 00:29:47.838596
random_50_trials_11_24_w_pruning.csv 0 days 00:14:58.388854
nsga_50_trials_11_20_w_pruning.csv 0 days 00:17:27.409901
pso_50_trials_11_20_w_pruning.csv 0 days 00:16:45.620683
harmony_50_trials_11_22_w_pruning.csv 0 days 00:21:20.889764
